# 计算机网络体系结构第一次作业

## 实验设置

### 买入

- 考虑买入证券交易费：过户费、佣金
- 过户费：按成交股票的金额 ×0.02‰收取，单位：元。双向收取（上海市场均收取）,过户费属于证券登记清算机构的收入
- 佣金：买卖双向收取，成交金额的 0.015%－0.3%，起点 5 元。此处取 0.018%

```python
deltaPositionValue = deltaPosition * stockToday["收盘"]
costBuy = deltaPositionValue*0.02/1e3 + max(deltaPositionValue*0.018/100, 5)
#self.cash -= deltaPosition * stockToday["收盘"]
self.cash = self.cash - deltaPositionValue - costBuy

#没有考虑self.cash不够，出现负数的情况
```

### 卖出

- 考虑买入证券交易费：印花税、过户费、佣金
- 印花税：单向收取，卖出成交金额的万分之五（5‱）印花税由政府收取
- 过户费：按成交股票的金额 ×0.02‰收取，单位：元。双向收取（上海市场均收取）,过户费属于证券登记清算机构的收入
- 佣金：买卖双向收取，成交金额的 0.015%－0.3%，起点 5 元。此处取 0.018%

```python
deltaPositionValue = pos["头寸"] * stockToday["收盘"]
costSell = deltaPositionValue*(1e-3 + 0.02/1e3) + max(deltaPositionValue*0.018/100, 5)
```


## 实验代码及说明

### 准备依赖

In [ ]:
!python -m pip install -q akshare pandas numpy

In [9]:
import akshare as ak
# import calendar
import datetime
# import numpy as np
import pandas as pd
import os

### 准备实验数据

In [10]:
stockNameCodes = [
    ("512200", "房地产ETF"),
    ("513050", "中概互联网ETF"),
    ("510300", "沪深300ETF"),
    ("510050", "上证50ETF"),
    ("510500", "中证500ETF"),
    ("159915", "创业板ETF"),
    ("159902", "中小100ETF"),
    ("512010", "医药ETF"),
    ("512880", "证券ETF"),
    ("512580", "环保ETF"),
    ("159920", "恒生ETF"),
    ("510900", "H股ETF"),
    ("513500", "标普500ETF"),
    ("513100", "纳指ETF"),
    ("512660", "军工ETF")
]


def Preprocess(startDate, endDate):
    # 获取标的历史数据
    startDate = startDate - datetime.timedelta(days=365)

    allStockDaily = {}
    path = f'data/{startDate.strftime("%Y")}-{endDate.strftime("%Y")}'
    if os.path.exists(path):
        print(f'使用已有数据({startDate.strftime("%Y")}-{endDate.strftime("%Y")})')
        for code, name in stockNameCodes:
            stockDaily = pd.read_csv(os.path.join(path, f"{name}_{code}.csv"))
            stockDaily["日期"] = pd.to_datetime(stockDaily["日期"])
            stockDaily = stockDaily.set_index("日期", drop=False)

            allStockDaily[code] = stockDaily
        return allStockDaily

    os.makedirs(path)
    for i, (code, name) in enumerate(stockNameCodes):
        print(f"{i}. {name}({code}) 正在读取数据...")
        try:
            # stockDaily = ak.stock_zh_a_hist(
            stockDaily = ak.fund_etf_hist_em(
                symbol=code,
                period="daily",
                start_date=startDate.strftime("%Y%m%d"),
                end_date=endDate.strftime("%Y%m%d"),
                adjust="hfq",
            )
        except:
            print(
                f'{name}({code}): {startDate.strftime("%Y-%m-%d")}--{endDate.strftime("%Y-%m-%d")}读取失败!'
            )
            continue
        # print(stockDaily.columns)
        stockDaily = stockDaily[stockDaily["开盘"].notnull()]
        stockDaily["日期"] = pd.to_datetime(stockDaily["日期"])

        stockDaily.to_csv(os.path.join(
            path, f"{name}_{code}.csv"), index=False)
        stockDaily = stockDaily.set_index("日期", drop=False)

        allStockDaily[code] = stockDaily
    return allStockDaily


if __name__ == "__main__":
    allStockDaily = Preprocess(datetime.datetime(
        2017, 9, 1), datetime.datetime(2024, 3, 14))
    print(f"共有{len(allStockDaily)}个标的数据")

使用已有数据(2016-2024)
共有15个标的数据
